In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import torchvision   
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import time
import torch.optim as optim
import os
import gdown
import zipfile
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)
!pwd

Thu Apr 29 22:12:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# data download source is not shown due to fair use

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(use_cuda, device, torch.cuda.device_count())

True cuda 1


## Model

In [ ]:
# We have looked at the following repository when coding this BetaVAE.
# @misc{Subramanian2020,
#   author = {Subramanian, A.K},
#   title = {PyTorch-VAE},
#   year = {2020},
#   publisher = {GitHub},
#   journal = {GitHub repository},
#   howpublished = {\url{https://github.com/AntixK/PyTorch-VAE}}
# }


class MyBetaVAE(nn.Module):
    def __init__(self, in_channels, z_dim, beta): # image should have size 64*64
        super(MyBetaVAE, self).__init__()
        self.z_dim = z_dim
        self.beta = beta

        # hidden_dims = [in_channels, 32, 64, 128, 256, 512]
        hidden_dims = [in_channels, 32, 64, 128, 256]
        self.encoder_final_size = 16
        self.hidden_dims = hidden_dims

        # encoder
        encoder_layers = []
        for i in range(len(hidden_dims)-1):
            encoder_layers.append(nn.Sequential(
                                        nn.Conv2d(hidden_dims[i], hidden_dims[i+1], kernel_size=3, stride=2, padding=1),
                                        nn.BatchNorm2d(hidden_dims[i+1]),
                                        nn.LeakyReLU()
                                  ))
        
        self.encoder = nn.Sequential(*encoder_layers)
        self.mu = nn.Linear(hidden_dims[-1] * self.encoder_final_size, z_dim)
        self.logvar = nn.Linear(hidden_dims[-1] * self.encoder_final_size, z_dim)

        # decoder
        self.decoder_in = nn.Linear(z_dim, hidden_dims[-1] * self.encoder_final_size)

        decoder_layers = []
        for i in range(len(hidden_dims)-1, 0, -1):
            decoder_layers.append(nn.Sequential(
                                      nn.ConvTranspose2d(hidden_dims[i], hidden_dims[i] if i == 1 else hidden_dims[i-1], kernel_size=3, stride=2, padding=1, output_padding=1),
                                      nn.BatchNorm2d(hidden_dims[i] if i == 1 else hidden_dims[i-1]),
                                      nn.LeakyReLU()
                                  ))
        
        self.decoder = nn.Sequential(*decoder_layers,
                                     nn.Conv2d(hidden_dims[1], hidden_dims[0], kernel_size=3, padding=1),
                                     nn.Sigmoid())
    
    def encode(self, x):
        out = self.encoder(x)
        out = torch.flatten(out, start_dim=1)
        mu = self.mu(out)
        logvar = self.logvar(out)
        self.mu_value = mu
        self.logvar_value = logvar
        self.x = x
        return mu, logvar
    
    def decode(self, x):
        out = self.decoder_in(x).view(-1, self.hidden_dims[-1], 4, 4)
        out = self.decoder(out)
        return out
    
    def reparam(self, mu, logvar):
        std = torch.exp(logvar / 2)
        epsilon = torch.autograd.Variable(torch.randn_like(std))
        return std * epsilon + mu


    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparam(mu, logvar)
        out = self.decode(z)
        self.x_cons = out
        return out, z
    
    def loss(self):
        reconstruction_loss = F.mse_loss(self.x_cons, self.x, reduction='sum')
        kl_div = torch.mean(torch.sum(-0.5 * (1 + self.logvar_value - self.mu_value ** 2 - self.logvar_value.exp()), dim=1), dim=0)
        return reconstruction_loss + self.beta * kl_div, reconstruction_loss, kl_div

In [ ]:
in_channels, z_dim, beta = 3, 128, 0.5
model = MyBetaVAE(in_channels, z_dim, beta)
model = model.to(device)
print(model, next(model.parameters()).is_cuda)

MyBetaVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
  )
  (mu): 

In [ ]:
learningRate = 0.0005
weightDecay = 0.0
epochs = 140
gamma = 0.95
torch.manual_seed(42)
np.random.seed(11785)

optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=weightDecay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=gamma, verbose=True)

Adjusting learning rate of group 0 to 5.0000e-04.


## DataLoader

In [ ]:
!mv /content/data /content/unique-142p

In [ ]:
!mkdir /content/data
!mkdir /content/data/unique-142p
!mv /content/unique-142p /content/data/unique-142p

In [ ]:
!ls /content/data

unique-142p


In [ ]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.Resize((64, 64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64, 64)),
    torchvision.transforms.ToTensor()                                              
])

def get_dataloaders(path, shuffle, portion, val_split, batch_size, test=False):
    dataset = torchvision.datasets.ImageFolder(root=path, transform=transforms)
    if test:
      test_dataset = torchvision.datasets.ImageFolder(root=path, transform=test_transforms)

    shuffle_dataset = shuffle
    use_proportion = portion
    validation_split = val_split
    batch_size = batch_size
    random_seed= 42

    # Creating data indices for training and validation splits:
    indices = list(range(len(dataset)))
    use_length = int(len(dataset) * use_proportion)
    split = int(np.floor((1 - validation_split) * len(dataset)) * use_proportion)
    if shuffle_dataset:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices = indices[:split]
    val_indices = indices[split:use_length]
    print(len(train_indices), len(val_indices))
    print(len(dataset), use_length, split)

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)
  

    train_dataloader = DataLoader(dataset, batch_size=batch_size, drop_last=True,sampler=train_sampler)
    val_dataloader = DataLoader(dataset, batch_size=batch_size, drop_last=True,sampler=val_sampler)

    if test:
      test_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True, sampler=train_sampler)
      return test_dataloader, test_dataloader
    

    return train_dataloader, val_dataloader

In [ ]:
train_loader, val_loader= get_dataloaders('/content/data', shuffle=True, portion=1.0, val_split=0.2, batch_size=256)

70389 17598
87987 87987 70389


## Train

In [ ]:
def train(model, train_loader, val_loader, epochs, folder=''):
    model.train()
    best_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        start_time = time.time()
        avg_loss = 0.0
        for batch_num, (x,y) in enumerate(tqdm(train_loader, position=0, leave=True)):
            x = x.to(device)
            
            optimizer.zero_grad()

            recons_x, z = model(x)
            loss, mse, kld = model.loss()

            with open(folder+'/train_loss.txt', 'a') as ft:
                ft.write('%f\n' % loss.item())
            with open(folder+'/train_mse_loss.txt', 'a') as ft1:
                ft1.write('%f\n' % mse.item())
            with open(folder+'/train_kld_loss.txt', 'a') as ft2:
                ft2.write('%f\n' % kld.item())
            
            loss.backward()
            optimizer.step()

            avg_loss += loss.item()
            

            if batch_num % 200 == 199:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/200))
                avg_loss = 0.0    
            
            torch.cuda.empty_cache()
            del x
            del recons_x
            del z
            del loss
            del mse
            del kld
            torch.cuda.empty_cache()

        end_time = time.time()
        print(end_time - start_time)
        val_loss = val(model, val_loader)
        scheduler.step()
        if val_loss < best_loss:
            best_loss = val_loss
            checkpoint = {"model": model.state_dict(), 
                          "optimizer": optimizer.state_dict(),
                          "scheduler": scheduler.state_dict()}
            torch.save(checkpoint, folder+'/v2_best_checkpoint_epoch'+str(epoch)+'.pt')
        if (epoch + 1) % 10 == 0:
            checkpoint = {"model": model.state_dict(), 
                          "optimizer": optimizer.state_dict(),
                          "scheduler": scheduler.state_dict()}
            torch.save(checkpoint, folder+'/v2_checkpoint_epoch'+str(epoch)+'.pt')
        print('Val Loss: {:.4f}'.format(val_loss))
        print('lr: ', optimizer.param_groups[0]['lr'])


def val(model, loader, folder=''):
    model.eval()
    avg_loss = 0.0
    avg_mse = 0.0
    avg_kld = 0.0

    with torch.no_grad():
        for batch_num, (x,y) in enumerate(tqdm(loader, position=0, leave=True)):
            x = x.to(device)
            _, _ = model(x)
            loss, mse, kld = model.loss()

            avg_loss += loss.item()  
            avg_mse += mse.item()
            avg_kld += kld.item()
            
            torch.cuda.empty_cache()
            del x
            del loss
            del mse
            del kld
            torch.cuda.empty_cache()

    model.train()
    avg_loss = avg_loss / len(loader)
    avg_mse = avg_mse / len(loader)
    avg_kld = avg_kld / len(loader)
    with open(folder+'/val_loss.txt', 'a') as fv:
        fv.write('%f\n' % avg_loss)
    with open(folder+'/val_mse_loss.txt', 'a') as fv1:
        fv1.write('%f\n' % avg_mse)
    with open(folder+'/val_kld_loss.txt', 'a') as fv2:
        fv2.write('%f\n' % avg_kld)

    return avg_loss

In [ ]:
train(model, train_loader, val_loader, epochs)

 73%|███████▎  | 200/274 [01:32<00:34,  2.15it/s]

Epoch: 1	Batch: 200	Avg-Loss: 3125383.4863


  0%|          | 0/68 [00:00<?, ?it/s]

126.82081341743469


  0%|          | 0/274 [00:00<?, ?it/s]

...

Epoch: 140	Batch: 200	Avg-Loss: 2939011.1550


  0%|          | 0/68 [00:00<?, ?it/s]

127.25381970405579


100%|██████████| 68/68 [00:28<00:00,  2.39it/s]

Adjusting learning rate of group 0 to 4.4872e-05.
Val Loss: 2934961.6581
lr:  4.487241596542534e-05
